In [28]:
# bibliotecas
from datetime import datetime, date
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import weekofyear


In [2]:
#criando um objeto sparksession object e um appName 
sparkSession=SparkSession.builder.appName("logs").getOrCreate()

In [3]:
# ler arquivo e carregar dados para um dataframe
df = sparkSession.read.option("header", "true").csv("website_logs.csv")

#df.head()

Row(user_id='10001', page_url='homepage.html', session_duration='15', date='2023-07-25')

In [6]:
# criar tabela temporária em memória com os dados para utilizar consulta SQL
df.createOrReplaceTempView("logs")
sparkSession.sql("select * from logs").show()

+-------+-----------------+----------------+----------+
|user_id|         page_url|session_duration|      date|
+-------+-----------------+----------------+----------+
|  10001|    homepage.html|              15|2023-07-25|
|  10002|product_page.html|             120|2023-07-25|
|  10003|    checkout.html|              45|2023-07-25|
|  10004|     contact.html|              20|2023-07-25|
|  10005|    homepage.html|              10|2023-07-25|
|  10006|product_page.html|              95|2023-07-25|
|  10007|        blog.html|             150|2023-07-26|
|  10008|    homepage.html|              25|2023-07-26|
|  10009|product_page.html|              85|2023-07-26|
|  10010|    checkout.html|              50|2023-07-26|
|  10011|         faq.html|              35|2023-07-27|
|  10012|    homepage.html|              12|2023-07-27|
|  10013|product_page.html|             110|2023-07-27|
|  10014|    checkout.html|              60|2023-07-27|
|  10015|        blog.html|             160|2023

In [14]:
# Ordenar páginas mais visitadas e selecionar as 10 primeiras
top_acessos = sparkSession.sql("select page_url, count(*) as counter from logs group by page_url order by 2 desc")
top_acessos.show(10)


+-----------------+-------+
|         page_url|counter|
+-----------------+-------+
|    homepage.html|     51|
|product_page.html|     46|
|    checkout.html|     35|
|     contact.html|     19|
|        blog.html|     18|
|         faq.html|     14|
|    about_us.html|      1|
+-----------------+-------+



In [24]:
# Calcular a média de duraçao das sessões de usuários
duracao_media_sessao = sparkSession.sql("select user_id, avg(session_duration) as media_sessao from logs group by 1 order by 1")
duracao_media_sessao.show()

+-------+------------------+
|user_id|      media_sessao|
+-------+------------------+
|  10001|19.666666666666668|
|  10002|124.33333333333333|
|  10003|48.666666666666664|
|  10004|24.333333333333332|
|  10005|14.333333333333334|
|  10006| 99.33333333333333|
|  10007|152.33333333333334|
|  10008|29.333333333333332|
|  10009| 87.66666666666667|
|  10010|54.666666666666664|
|  10011|37.666666666666664|
|  10012|14.333333333333334|
|  10013|114.66666666666667|
|  10014|62.666666666666664|
|  10015|             161.0|
|  10016|20.666666666666668|
|  10017|104.33333333333333|
|  10018|57.666666666666664|
|  10019|              26.0|
|  10020|              18.0|
+-------+------------------+
only showing top 20 rows



In [53]:
# Como descobrir a quantidade de usuários que já acessaram o site mais de uma vez numa mesma semana:

# adiciona uma coluna com o número da semana - aqui só funciona assim porque as datas de acesso são todas do mesmo ano
# se fossem em anos diferentes, seria interessante combinar o número da semana com o ano para gerar uma "chave"
logs_semanas = df.withColumn('week',weekofyear(df.date))

#cria tabela virtual para acessar com sql
logs_semanas.createOrReplaceTempView("log_semana")

# Agrupa quantidade de acessos por usuário por semana, e deixa apenas usuários que acessaram mais de uma vez numa mesma semana e em qual semana isso aconteceu
acessos_semanais_por_usuario = sparkSession.sql("select user_id, count(*) as weekly_access from log_semana group by user_id, week having count(*)>1 order by 1")

# Agrupa os usuários e conta em quantas semanas eles acessaram o site mais de uma vez
acessos_semanais_por_usuario.groupby("user_id").count().show()

# Usando a função describe no código da linha anterior, é possível obter a quantidade de usuários distintos que já acessaram o site mais de uma vez por semana
acessos_semanais_por_usuario.groupby("user_id").count().describe().show()

+-------+-----+
|user_id|count|
+-------+-----+
|  10039|    2|
|  10038|    2|
|  10001|    1|
|  10023|    1|
|  10040|    2|
|  10022|    1|
|  10002|    1|
|  10035|    3|
|  10028|    1|
|  10033|    2|
|  10031|    2|
|  10027|    1|
|  10036|    2|
|  10037|    2|
|  10024|    1|
|  10026|    1|
|  10029|    1|
|  10032|    3|
|  10034|    2|
|  10030|    2|
+-------+-----+
only showing top 20 rows

+-------+------------------+------------------+
|summary|           user_id|             count|
+-------+------------------+------------------+
|  count|                21|                21|
|   mean|           10028.0| 1.619047619047619|
| stddev|10.454664030948324|0.6690433824641325|
|    min|             10001|                 1|
|    max|             10040|                 3|
+-------+------------------+------------------+

